In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from modules.utils.visualization import (
    plot_samples
)
from IPython.display import display
from pathlib import Path


pl.seed_everything(1234)

# ------------
# dataset path
# ------------
dataset_root_path = Path('/root/data/processed/drive_and_act/')
keypoint_2d_path = dataset_root_path / 'keypoint_detection_results'
keypoint_3d_path = dataset_root_path / 'annotations'
# ------------
# model
# ------------
image_width = 1280
image_height = 1024
batch_size = 64
max_epoch = 200
val_check_period = 5
early_stopping_patience = 5
# ------------
# saved model path
# ------------
pretrained_model_path = './saved_lifter_2d_3d_model/synthetic_cabin_ir/A_Pillar_Codriver/prediction/linear_model/'
with open(f'{pretrained_model_path}/best_model_path.txt', 'r') as f:
    checkpoint_path = f.readline()


In [ ]:
def run_experiment(
    train_actors,
    val_actors,
    test_actors,
    is_plot_gt_skeleton=False,
    model_suffix=''
):
    
    train_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=train_actors,
        exclude_ankle=True,
        exclude_knee=True
    )
    val_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=val_actors,
        exclude_ankle=True,
        exclude_knee=True
    )
    test_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=test_actors,
        exclude_ankle=True,
        exclude_knee=True
    )

    print(
        'train_dataset', len(train_dataset),
        'val_dataset', len(val_dataset),
        'test_dataset', len(test_dataset)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=24)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True, num_workers=24)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=24)

    all_activities = train_dataset.activities.union(val_dataset.activities).union(test_dataset.activities)
    lit_model = LitSimpleBaselineLinear(exclude_ankle=True, exclude_knee=True, all_activities=all_activities)
    lit_model = lit_model.load_from_checkpoint(checkpoint_path)
    model_checkpoint = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=1)
    early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=early_stopping_patience)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    saved_model_path = f'./saved_lifter_2d_3d_model/drive_and_act/transfer_learning/synthetic_cabin_ir/one_actors/linear_model_{model_suffix}'
    if not os.path.exists(saved_model_path):
        os.makedirs(saved_model_path)
    trainer = pl.Trainer(
        # max_steps=10,
        max_epochs=max_epoch,
        callbacks=[model_checkpoint, early_stopping],
        accelerator=device,
        check_val_every_n_epoch=val_check_period,
        default_root_dir=saved_model_path,
        gradient_clip_val=1.0
    )
    trainer.fit(lit_model, train_loader, val_loader)
    with open(f'{saved_model_path}/best_model_path.txt', 'w') as f:
        f.writelines(model_checkpoint.best_model_path)
    best_checkpoint_path = model_checkpoint.best_model_path
    trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)
    # plot_samples(
    #     dataset_root_path,
    #     trainer.model,
    #     test_loader,
    #     'test',
    #     img_figsize=(20, 10),
    #     img_width=image_width,
    #     img_height=image_height,
    #     plot_figsize=(20.5, 10),
    #     sample_idices=[1000, 2500, 6000],
    #     is_plot_gt_skeleton=is_plot_gt_skeleton
    # )
    return (
        lit_model.test_history[0]['mpjpe'],
        lit_model.test_history[0]['pjpe'],
        lit_model.test_history[0]['activities_mpjpe']
    )


## Fine-Tuned with 1 Actors

In [ ]:
all_mpjpe = []
all_pjpe = []
all_activity_mpjpe = []
all_actors = ['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8']
for vp in ['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8']:
    display(f'Actor: {vp}')
    mpjpe, pjpe, activities_mpjpe = run_experiment(
        train_actors=[vp],
        val_actors=['vp9', 'vp10', 'vp15'],
        test_actors=['vp11', 'vp12', 'vp13', 'vp14'],
        is_plot_gt_skeleton=False,
        model_suffix=vp
    )
    all_mpjpe.append(mpjpe)
    all_pjpe.append(pjpe)
    all_activity_mpjpe.append(activities_mpjpe)

In [ ]:
pd.DataFrame(all_mpjpe, index=all_actors)

In [ ]:
pd.DataFrame(all_mpjpe, index=all_actors).mean()

In [ ]:
pjpe_df = pd.concat(all_pjpe, axis=1)
pjpe_df.columns = all_actors
pjpe_df

In [ ]:
pd.DataFrame(pjpe_df.mean(axis=1), columns=['average_pjpe'])

In [ ]:
all_activity_mpjpe[0]

In [ ]:
pd.DataFrame(all_activity_mpjpe, index=all_actors).T.sort_index()

In [ ]:
pd.DataFrame(pd.DataFrame(all_activity_mpjpe, index=all_actors).T.mean(axis=1), columns=['pjpe']).sort_index()